In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KernelDensity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import norm
from scipy.stats import multivariate_normal
import aa_utils


ModuleNotFoundError: No module named 'aa_utils'

<font  style="font-size: 4rem; color: #1abc9c"> Bayesian Learning </font>

## Table of Contents
* [Exercise 1: Gaussian Naive Bayes](#exo1)
    * [Naive Bayes exploration](#section_1_1)
    * [Visualization](#section_1_2)
    * [Comparison with KDE](#section_1_3)
    * [Evaluation](#section_1_4)
* [Exercise 2: k-Nearest Neighbors](#exo2)
    * [Basic k-NN](#section_2_1)
    * [Data preprocessing](#section_2_2)
    * [Select $k$ with a validation set](#section_2_3)
* [Appendix: KDE approfondissement](#exo3)
    * [1D KDE (Naive Bayes hypothesis)](#section_3_1)
    * [2D KDE (Multivariate KDE)](#section_3_2)
    * [K-nn cleaning and condensing](#section_3_2)
    


# <font color="#1E90FF">Exercise 1: Gaussian Naive Bayes</font><a class="anchor" id="exo1"></a>

In Scikit-learn, the Gaussian Naive Bayes algorithm for classification is implemented via the <code>GaussianNB</code> class. 

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB

In [ ]:
df = pd.read_csv('./data/data_exam.txt', sep = ' ')
print(df)
dataset = df.rename(columns={df.columns[0]: 'X1',df.columns[1]: 'X2',df.columns[2]: 'Y'})
sns.scatterplot(data=dataset, x='X1', y='X2', hue='Y', marker='+', palette=['blue','red'])

In [ ]:
# Dataset split into train/test set
data_train, data_test = train_test_split(dataset, test_size = 0.3, random_state = 6)

## <font color="#9400D3">1. Gaussian Naive Bayes Exploration</font><a class="anchor" id="section_1_1"></a>

We learn a Gaussian Naive Bayes classifier on the training data, and analyze the model.

In [ ]:
# learn a Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(data_train.loc[:, ['X1', 'X2']], data_train.Y)
print('Probability of each class:', gnb.class_prior_)
print('Variance of each feature per class:',gnb.sigma_)
print('Mean of each feature per class:',gnb.theta_)

We plot the pdf (probability density function) for $x_1|w_1$ and $x_1|w_2$.

<font color="blue">**TODO:**</font> : Plot the pdf for $x_2|w_1$ and $x_2|w_2$.

In [ ]:
np_note1_w1 = data_train[data_train.Y == 0].iloc[:, 0].to_numpy()
np_note1_w2 = data_train[data_train.Y == 1].iloc[:, 0].to_numpy()

x = np.linspace(0, 120, 100)
# Plot density for class w1=0
density_w1 = norm.pdf(x, gnb.theta_[0, 0], np.sqrt(gnb.sigma_[0, 0]))
plt.plot(x, density_w1, color = "blue", linestyle = "-",label='gaussian class 0')
plt.hist(np_note1_w1, density=True, color='blue', alpha=0.2, label='hist class 0')
plt.scatter(np_note1_w1, np.zeros(len(np_note1_w1)), color = "blue")
# Plot density for class w2=1
density_w2 = norm.pdf(x, gnb.theta_[1, 0], np.sqrt(gnb.sigma_[1, 0]))
plt.plot(x, density_w2, color = "red", linestyle = "-",label='gaussian class 1')
plt.hist(np_note1_w2, density=True, color='red', alpha=0.2, label='hist class 1')
plt.scatter(np_note1_w2, np.zeros(len(np_note1_w2)), color = "red")
plt.legend(loc='upper right')

In [ ]:
# TODO - Plot for x2



<font color="red">**Question 1:**</font> Compute the class of the first example of the test set. **You will detail the calculation in your report, indicating the formulas (math) used**, before giving their numerical values.

**Tips**:
- $P(x_i|w_j)$ is given by <code>norm.pdf</code>($x_i, \mu_j, \sigma_j$). Check the given code above that plots the densities.
- You can verify your computations using the fonction <code>predict_proba</code>. 

Usage : <code>gnb.predict_proba(data_test.iloc[0,:2].values.reshape(1, -1))</code>
- To end up with the same values, you should compute the 'real' value of $P(w|x)$, which means normalizing by $P(x)$, even if this is not needed in practice to classify the data.

Remember your probas: $P(x)=P(x|w_1) \times P(w_1) + P(x|w_2) \times P(w_2)$

In [ ]:
# TO DO - 'manually' compute the class of the first example in the test set




## <font color="#9400D3">2. Vizualisation</font><a class="anchor" id="section_1_2"></a>

In [ ]:
# Plot the gaussian NB likelihood
aa_utils.draw_gaussianNB_likelihood(gnb,data_train)

In [ ]:
# Plot the NB classifier decision boundaries
aa_utils.draw_decision_boundaries(gnb,data_train)

In [ ]:
# plot Naive Bayes predicted probabilities for class w2
aa_utils.draw_predicted_probabilities(gnb,data_train)

<font color="red">**Question 2:**</font> For each of the figures below explain what is represented. You will indicate in each case the corresponding mathematical formula.


## <font color="#9400D3">3. Comparison with KDE</font><a class="anchor" id="section_1_3"></a>

We now compute the likelihood by Kernel Density Estimation with a 'gaussian' kernel implemented in scikit-learn by <code>KernelDensity</code> class.

In [ ]:
# Plot density for class w1=0
gkde_w1 = KernelDensity(kernel='gaussian', bandwidth=5).fit(np_note1_w1.reshape(-1, 1))
g_log_dens_w1 = gkde_w1.score_samples(x.reshape(-1, 1))
plt.plot(x,np.exp(g_log_dens_w1), color="blue",linestyle="--", label='gaussian kde class 0')
plt.hist(np_note1_w1, density=True, color='blue', alpha=0.2, label='hist class 0')
plt.scatter(np_note1_w1, np.zeros(len(np_note1_w1)), color = "blue")
# Plot density for class w2=1
gkde_w2 = KernelDensity(kernel='gaussian', bandwidth=5).fit(np_note1_w2.reshape(-1, 1))
g_log_dens_w2 = gkde_w2.score_samples(x.reshape(-1, 1))
plt.plot(x,np.exp(g_log_dens_w2), color="red",linestyle="--", label='gaussian kde class 1')
plt.hist(np_note1_w2, density=True, color='red', alpha=0.2, label='hist class 1')
plt.scatter(np_note1_w2, np.zeros(len(np_note1_w2)), color = "red")
plt.legend(loc='upper right')

<font color="red">**Question 3:**</font> 
Explain the difference between the estimation of $P(x|w)$ by a gaussian pdf, and the gaussian KDE.

<font color="red">**Question 4:**</font> Change the value of the <code>bandwith</code> parameter (give a float value). Conclude on the influence of this parameter.

Rather than assuming attributes independance and use 1D density estimation, KDE can advantageously be used on all the attributes. In our case, the kernel used is a 2D gaussian parametrized by the bandwith $h$.

<font color="blue">**TODO:**</font> Observe below the difference with the likelihood estimated by the gaussian NB. You can change the bandwith parameter value to see the effects of this hyper-parameter.

In [ ]:
# Plot Multivariate KDE
# bw_h is the bandwith parameter
aa_utils.draw_sk_kde_densities(bw_h=9,data=data_train)

## <font color="#9400D3">4. Evaluation</font><a class="anchor" id="section_1_4"></a>

There is no implementation in scikit-learn of a Bayes Classifier using KDE.

<font color="blue">**TODO:**</font> Implement the KDE-based Bayes classifier.

**Hints:** Following methods will be usefull:
- <code>kde.score_samples(X)</code> : compute the log-likelihood of each sample under the model
- to get the likelihood: <code>np.exp(kde.score_samples(X))</code>

where <code>kde</code> is the Kernel Density estimator (model).

<font color="red">**Question 5:**</font> Compare **in a table** the performances in generalization of the gaussian NB and a KDE-based Bayes classifier in terms of accuracy or error, for different values of $h$ and different kernels.

<code>KernelDensity</code> implements several common kernel forms. Refer to the documentation: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html

In [ ]:
# TODO - Performance of NB 


In [ ]:
# TODO - Bayes classification with 2D KDE and evaluation


# <font color="#1E90FF">Exercise 2: k-Nearest Neighbors</font><a class="anchor" id="exo2"></a>

## <font color="#9400D3">1. Basic k-NN</font>

We will first look at the k-nn classifier with $k=1$.


In [ ]:
# load data
df = pd.read_csv('./data/knn_dataset.csv', sep = ',')
print(df)
sns.scatterplot(data=df, x='X1', y='X2', hue='Y', marker='+', palette=['blue','red'])

In [ ]:
# Split data
data_train, data_test = train_test_split(df, test_size = 0.3, random_state = 13)
data_valid, data_test = train_test_split(data_test, test_size = 0.5, random_state = 13)

In [ ]:
# k-NN
nn_clf = KNeighborsClassifier(n_neighbors=1) # this initializes a knn for k = 1
nn_clf.fit(data_train.iloc[:,:-1],data_train.Y)

<font color="red">**Question 1:**</font> Which distance among samples is used to compute the nearest neighobors?

In [ ]:
# Affichage frontiere de decision
aa_utils.draw_decision_boundaries(nn_clf,data_test)

In [ ]:
# Knn classifier evaluation
print('Accuracy test sample :', nn_clf.score(data_test.iloc[:,:-1], data_test.Y))

## <font color="#9400D3">2. Data preprocessing</font><a class="anchor" id="section_2_2"></a>

When classifiers are based on distance calculations between examples, it may be important to preprocess the data.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
np_scaled=scaler.fit_transform(df.iloc[:,:-1])

df_scaled = pd.DataFrame(np_scaled, columns=['X1','X2'])
df_scaled['Y'] = df.Y
sns.scatterplot(data=df_scaled, x='X1', y='X2', hue='Y', marker='+', palette=['blue','red'])

<font color="red">**Question 2:**</font> What is the effect of the pre-processing applied here?

<font color="red">**Question 3:**</font> 
- Compare the performance of the KNN classifier using pre-processed data with the previous one. 
- Explain why the classifier benefits from the pre-processing.

In [ ]:
# TODO - Evaluate the KNN classifier with pre-processed data


In [ ]:
# Affichage frontiere de decision
#aa_utils.draw_decision_boundaries(model,data)

## <font color="#9400D3">3. Select $k$ with a validation set</font><a class="anchor" id="section_2_3"></a>

$k$ being an hyper-parameter of the KNN classifier, its value should be tuned using a validation set.

<font color="blue">**TODO:**</font> 
- Plot the validation error as a function of $k$
- Select the best value of $k$
- Evaluate the resulting classifier
- Plot the decision boundaries

In [ ]:
# TODO - Plot the validation error as a function of the number of neighbors.


In [ ]:
# Find the best hyper-parameter k


# Evaluate the classifier


In [ ]:
# Affichage frontiere de decision
#aa_utils.draw_decision_boundaries(...)

<font color="red">**Question 4:**</font> Compare the performances of the (gaussian) Naive Bayes and the (tuned) k-NN classifiers.


In [ ]:
# TODO - NB and k-NN classifiers comparison

